In [ ]:
from utilitaires import *

import numpy as np
import json
import uuid
from IPython.display import HTML

# 1. Load the MNIST image
img = d

# Convert to plain ints for JSON
data = img.astype(int).tolist()

# Generate a short unique suffix
suffix = uuid.uuid4().hex[:8]

html = f"""
<style>
  .grid-{suffix} {{
    display: grid;
    grid-template-columns: repeat(3, auto);
    grid-auto-rows: auto;
    grid-gap: 10px;
    align-items: center;
    text-align: center;
  }}
  .grid-{suffix} canvas {{
    image-rendering: pixelated;
    width: 280px;
    height: 280px;
    border: 1px solid #ccc;
  }}
  .controls-v-{suffix}, .controls-h-{suffix} {{
    grid-column: 1 / -1;
    text-align: center;
    margin: 5px 0;
  }}
</style>

<div class="grid-{suffix}">
  <!-- Row 0: v-slider above first row -->
  <div class="controls-v-{suffix}">
    v: <input type="range" id="v-{suffix}" min="1" max="26" value="15">
  </div>

  <!-- Row 1: headers -->
  <div>Original</div><div>Filter</div><div>Result</div>

  <!-- Row 2: first row of canvases -->
  <canvas id="c0-{suffix}" width="28" height="28"></canvas>
  <canvas id="c1-{suffix}" width="28" height="28"></canvas>
  <canvas id="c2-{suffix}" width="28" height="28"></canvas>

  <!-- Row 3: h-slider above second row -->
  <div class="controls-h-{suffix}">
    h: <input type="range" id="h-{suffix}" min="1" max="26" value="8">
  </div>

  <!-- Row 4: second row of canvases -->
  <canvas id="c3-{suffix}" width="28" height="28"></canvas>
  <canvas id="c4-{suffix}" width="28" height="28"></canvas>
  <canvas id="c5-{suffix}" width="28" height="28"></canvas>
</div>

<script>
  (function() {{
    const img = {json.dumps(data)};
    const suf = "{suffix}";

    function makeFilter(v, h, vertical) {{
      const F = Array.from({{length:28}}, () => Array(28).fill(0));
      for (let i=0; i<28; i++) for (let j=0; j<28; j++) {{
        if (vertical) {{
          if (j===v)      F[i][j]=1;
          if (j===v-1||j===v+1) F[i][j]=-1;
        }} else {{
          if (i===h)      F[i][j]=1;
          if (i===h-1||i===h+1) F[i][j]=-1;
        }}
      }}
      return F;
    }}

    function draw(ctx, M, type) {{
      const imgData = ctx.createImageData(28,28);
      let min=Infinity, max=-Infinity;
      if (type==='result') {{
        M.forEach(r=>r.forEach(p=>{{const v=Math.abs(p); min=Math.min(min,v); max=Math.max(max,v);}}));
      }}
      for (let i=0; i<28; i++) for (let j=0; j<28; j++) {{
        const idx=(i*28+j)*4;
        let r,g,b;
        if (type==='orig') {{
          const v=M[i][j]; r=g=b=v;
        }} else if (type==='filter') {{
          const val=M[i][j];
          if      (val>0) {{ r=0;   g=0;   b=Math.round(val*255); }}
          else if (val<0) {{ r=Math.round(-val*255); g=0; b=0; }}
          else            {{ r=g=b=0; }}
        }} else {{
          let v=Math.abs(M[i][j]);
          if (max>min) v=Math.round(255*(v-min)/(max-min));
          r=g=b=v;
        }}
        imgData.data[idx  ]=r;
        imgData.data[idx+1]=g;
        imgData.data[idx+2]=b;
        imgData.data[idx+3]=255;
      }}
      ctx.putImageData(imgData,0,0);
    }}

    function updateAll() {{
      const v = +document.getElementById("v-"+suf).value;
      const h = +document.getElementById("h-"+suf).value;
      const ctxs = [];
      for (let i=0; i<6; i++) {{
        ctxs.push(document.getElementById("c"+i+"-"+suf).getContext("2d"));
      }}

      // first row
      draw(ctxs[0], img,        'orig');
      const fv = makeFilter(v,h,true);
      draw(ctxs[1], fv,         'filter');
      const rv = img.map((r,i)=>r.map((p,j)=>p*fv[i][j]));
      draw(ctxs[2], rv,         'result');

      // second row
      draw(ctxs[3], img,        'orig');
      const fh = makeFilter(v,h,false);
      draw(ctxs[4], fh,         'filter');
      const rh = img.map((r,i)=>r.map((p,j)=>p*fh[i][j]));
      draw(ctxs[5], rh,         'result');
    }}

    document.getElementById("v-"+suf).oninput = updateAll;
    document.getElementById("h-"+suf).oninput = updateAll;
    updateAll();
  }})();
</script>
"""

display(HTML(html))
